In [21]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import dm4bem

In [22]:
start_date = '02-01 12:00:00'
end_date = '02-07 18:00:00'

In [23]:
start_date = '2000-' + start_date
end_date = '2000-' + end_date
print(f'{start_date} \tstart date')
print(f'{end_date} \tend date')

2000-02-01 12:00:00 	start date
2000-02-07 18:00:00 	end date


In [24]:
filename = './weather_data/EnergyPlusNantes.epw'
[data, meta] = dm4bem.read_epw(filename, coerce_year=None)
weather = data[["temp_air", "dir_n_rad", "dif_h_rad"]]
del data

In [25]:
weather.index = weather.index.map(lambda t: t.replace(year=2000))
weather = weather.loc[start_date:end_date]

In [26]:
# Temperature sources
To = weather['temp_air']

In [27]:
# total solar irradiance
wall_out = pd.read_csv('./walls_out.csv')
w0 = wall_out[wall_out['ID'] == 'w0']

surface_orientation = {'slope': w0['β'].values[0],
                       'azimuth': w0['γ'].values[0],
                       'latitude': 45}

rad_surf = dm4bem.sol_rad_tilt_surf(
    weather, surface_orientation, w0['albedo'].values[0])

In [28]:
Etot = rad_surf.sum(axis=1)

In [29]:
# solar radiation absorbed by the outdoor surface of the wall
Φo = w0['α1'].values[0] * w0['Area'].values[0] * Etot

In [30]:
# window glass properties
α_gSW = 0.38    # short wave absortivity: reflective blue glass
τ_gSW = 0.30    # short wave transmitance: reflective blue glass
S_g = 9         # m², surface area of glass

# solar radiation absorbed by the indoor surface of the wall
Φi = τ_gSW * w0['α0'].values[0] * S_g * Etot

In [31]:
# solar radiation absorbed by the glass
Φa = α_gSW * S_g * Etot

In [32]:
# indoor air temperature set-point
Ti_sp = pd.Series(20, index=To.index)

In [33]:
Ti_day, Ti_night = 20, 16

Ti_sp = pd.Series(
    [Ti_day if 6 <= hour <= 22 else Ti_night for hour in To.index.hour],
    index=To.index)

In [34]:
# auxiliary (internal) sources
Qa = 0 * np.ones(weather.shape[0])

In [35]:
# Input data set
input_data_set = pd.DataFrame({'To': To, 'Ti_sp': Ti_sp,
                               'Φo': Φo, 'Φi': Φi, 'Qa': Qa, 'Φa': Φa,
                               'Etot': Etot})
input_data_set.to_csv('./toy_model/input_data_set.csv')

OSError: Cannot save file into a non-existent directory: 'toy_model'